In [1]:
import numpy as np
import nibabel
import nilearn
from nilearn.image import resample_to_img
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.plotting import show
from nilearn.plotting import plot_roi
from nilearn import image
from nilearn.masking import apply_mask
# get_ipython().magic('matplotlib inline')
import scipy
import matplotlib
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.input_data import NiftiMasker
#from nilearn import plotting
import nibabel
from nilearn.masking import apply_mask
from nilearn.image import load_img
from nilearn.image import new_img_like
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, svm, metrics
from sklearn.linear_model import Ridge
from sklearn.svm import SVC, LinearSVC
from sklearn.cross_validation import KFold
from sklearn.cross_validation import LeaveOneLabelOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_selection import SelectFwe
from scipy import signal
from scipy.fftpack import fft, fftshift
from scipy import interp

params = {'legend.fontsize': 'large',
          'figure.figsize': (5, 3),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
font = {'weight': 'bold',
        'size': 22}
plt.rc('font', **font)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile, f_classif, GenericUnivariateSelect, SelectKBest, chi2
from sklearn.feature_selection import RFE
import os
import seaborn as sns
import pandas as pd
import csv

/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
projectDir='/jukebox/norman/amennen/prettymouth/'
DMNmask='/jukebox/norman/amennen/MNI_things/Yeo_JNeurophysiol11_MNI152/Yeo_Network7mask_reoriented_resampledBOLD.nii.gz'
fmriprep_dir=projectDir + '/derivatives/fmriprep'

In [3]:
# load subject numbers
subInd = 0
nsub=38
allnames = []
allgroups = []
groupInfo={}
# skip subjects 039 and 116
with open(projectDir + 'participants.tsv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        if 'sub' in row[0]:
            # now skip the subjects we don't want to analyze
            allInfo = row[0].split('\t')
            subjName=allInfo[0]
            if subjName != 'sub-039' and subjName != 'sub-116':
                if allInfo[3] == 'paranoia':
                    group = 0
                elif allInfo[3] == 'affair':
                    group = 1
                allnames.append(subjName)
                allgroups.append(group)
                subInd+=1

In [4]:
paranoidSubj = allnames[0:19]
cheatingSubj = allnames[19:]
paranoidLabel = allgroups[0:19]
cheatingLabel = allgroups[19:]
nfolds=19

In [15]:
np.shape(masked_data_zscore)

(475, 3757)

In [5]:
# now loop through all subjects for training/testing
# train model on 2 
# for each loop, leave 2 subjects out, train on 36 (18/group) --> 19 fold classifier
sindex=0
data_as_vector = np.zeros((nsub,475*3757))
for s in np.arange(nsub):
    subjName=allnames[s]
    subjData=fmriprep_dir + '/' + subjName + '/' + 'func' + '/' + subjName + '_task-prettymouth_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    masked_data = apply_mask(subjData, DMNmask)
    masked_data_zscore = scipy.stats.zscore(masked_data,axis=0)
    if np.any(np.std(masked_data,axis=0)==0):
        masked_data_zscore[:,np.std(masked_data,axis=0)==0] = 0
    data_vectorized = masked_data_zscore.flatten()
    data_as_vector[s,:] = data_vectorized
    # zscore data first

/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/jukebox/pkgs/PYGER/be

In [6]:
# try first way of converting everyone to a vector
accuracy = np.zeros(nfolds)
for f in np.arange(nfolds):
    testingSubjects = np.array([paranoidSubj[f],cheatingSubj[f]])
    testingLabels = np.array([paranoidLabel[f],cheatingLabel[f]])
    indTrain = np.arange(19)
    trainingSubjectsInd=np.concatenate([indTrain[:f],indTrain[f+1:]])
    #trainingSubjects=[paranoidSubj[j] for j in trainingSubjectsInd]+[cheatingSubj[j] for j in trainingSubjectsInd]
    trainingLabels = [paranoidLabel[j] for j in trainingSubjectsInd]+[cheatingLabel[j] for j in trainingSubjectsInd]
    s1 = np.arange(19)
    s2 = np.arange(19) + 19
    trainingInd = [s1[j] for j in trainingSubjectsInd]+[s2[j] for j in trainingSubjectsInd]
    clf = LinearSVC()
    clf.fit(data_as_vector[trainingInd,:],trainingLabels)
    accuracy[f] = clf.score(data_as_vector[testingInd,:],testingLabels)
    print(f)
    print(accuracy[f])

0
0.5
1
0.5
2
0.5
3
1.0
4
1.0
5
0.5
6
1.0
7
1.0
8
0.5
9
1.0
10
0.5
11
1.0
12
0.5
13
0.5
14
1.0
15
0.5
16
1.0
17
0.5
18
1.0


In [10]:
mean_acc = np.mean(accuracy)
print('mean accuracy is %2.2f +/- %2.2f' % (mean_acc,scipy.stats.sem(accuracy)))
# first try: 73.6 % accuracy

mean accuracy is 0.74 +/- 0.06


In [14]:
type(data_as_vector)

numpy.ndarray

In [143]:
testingInd

array([ 1, 20])